# Redes Neuronales - Trabajo Práctico N° 1 - Ejercicio 2 - Notebook #2
En esta segunda notebook, se busca definir cuál métrica es más apropiada para analizar la performance del modelo y qué hiper parámetros se van a utilizar para el ajuste del modelo acorde a la validación. Finalmente, estas decisiones se vuelcan en la selección del mejor modelo para el problema de la clasificación de correos electrónicos asociados grupos de noticias.

### Fuentes útiles
* https://en.wikipedia.org/wiki/Bessel%27s_correction
* https://en.wikipedia.org/wiki/Kernel_density_estimation
* https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation
* https://stackoverflow.com/questions/58046129/can-someone-give-a-good-math-stats-explanation-as-to-what-the-parameter-var-smoo
* https://scikit-learn.org/stable/modules/density.html

### Integrantes del grupo
* Gaytan, Joaquín Oscar
* Kammann, Lucas Agustín

# 1. Métrica
La métrica a utilizar para cuantificar la performance de los modelos, seleccionar los hiperparámetros y validarlos, será la **sensibilidad** o **recall**.

## 1.1. Justificación
Se emplea el recall o sensibilidad respecto de los positivos, que se calcula como:
$$recall = \frac{TP}{TP+FN}$$
Es decir, esta métrica da información sobre la proporción de positivos identificados sobre el total de positivos (reales). En el caso del diagnóstico de una enfermedad, nos interesa minimizar el número de falsos negativos, dado que una persona clasificada como negativo pero que efectivamente esté enferma puede no recibir el tratamiento correspondiente, empeorando su cuadro y poniendo 

# 2. Preparación de los datasets

## 2.1. Cargando el dataset original

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read database from .csv
df = pd.read_csv('../assets/diabetes.csv', delimiter=',')

## 2.2. Sustitución de valores nulos
Los valores nulos de las variables o características para la clasificación se reemplazan por NaN o Not a Number, para evitar que sean procesados en el análisis estadístico posterior, de esta forma luego serán reemplazados por algún estadístico.

In [3]:
# Filtering Glucose values
df['Glucose'].replace(0, np.nan, inplace=True)

# Filtering Blood Pressure values
df['BloodPressure'].replace(0, np.nan, inplace=True)

# Filtering Skin Thickness values
df['SkinThickness'].replace(0, np.nan, inplace=True)

# Filtering Insulin values
df['Insulin'].replace(0, np.nan, inplace=True)

# Filtering Body Mass Index values
df['BMI'].replace(0, np.nan, inplace=True)

## 2.3. Filtrado de outliers

In [4]:
from src.helper import remove_outliers

In [5]:
for column in df.columns:
    remove_outliers(df, column)

## 2.4. Separación de datasets
Se separa el dataset original en los datasets de train, valid y test. Además, se debe corregir que los valores inválidos del dataset original fueron reemplazados por el valor NaN.

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
# Splitting into the total train and the test datasets, because
# the total train contains the train and valid datasets used for
# hiper parameter selection
train, test = train_test_split(df, test_size=0.2, random_state=27)

## 2.5. Sustitución de valores inválidos
Se filtran los valores inválidos de cada una de las variables, y se los reemplaza utilizando la media obtenida en el conjunto de entrenamiento. Particularmente, se opta por emplear la media de todo el conjunto de entrenamiento, para no introducir sesgo esencialmente dentro del conjunto empleado para la evaluación del modelo.

In [8]:
# Compute the mean of training
train_means = train.mean().to_numpy()

# Replacing nan values of the test dataset with training mean values
for index, column in enumerate(train.columns):
    train.loc[:,column].replace(np.nan, train_means[index], inplace=True)

# Replacing nan values of the test dataset with training mean values
for index, column in enumerate(test.columns):
    test.loc[:,column].replace(np.nan, train_means[index], inplace=True)

C:\anaconda\lib\site-packages\pandas\core\series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


## 2.6. Conjuntos de entrenamiento y evaluación

In [9]:
# Extracting the inputs and outputs of the train dataset
x_train = train.to_numpy()[:,:8]
y_train = train.to_numpy()[:,8]

# Extracting the inputs and outputs of the test dataset
x_test = test.to_numpy()[:,:8]
y_test = test.to_numpy()[:,8]

In [10]:
train.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000
mean,3.859247,122.032787,72.069808,28.893519,135.921311,32.366556,0.431246,32.792763,0.356678
std,3.308473,30.333958,10.708903,8.288153,52.552469,6.605536,0.244934,10.988938,0.479409
min,0.000000,44.000000,40.000000,7.000000,14.000000,18.200000,0.084000,21.000000,0.000000
25%,1.000000,99.000000,65.000000,25.000000,122.000000,27.500000,0.238000,24.000000,0.000000
50%,3.000000,118.000000,72.069808,28.893519,135.921311,32.366556,0.381500,29.000000,0.000000
75%,6.000000,141.000000,78.000000,32.000000,135.921311,36.800000,0.585250,40.000000,1.000000
max,13.000000,199.000000,104.000000,54.000000,360.000000,50.000000,1.189000,66.000000,1.000000


In [11]:
test.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,154.000000,154.000000,154.000000,154.000000,154.000000,154.000000,154.000000,154.000000,154.000000
mean,3.499086,120.318395,72.282847,28.927850,127.967511,31.575970,0.424461,32.853106,0.318182
std,3.106370,30.901014,11.544378,8.147233,46.974638,5.539948,0.245579,11.315674,0.467290
min,0.000000,56.000000,44.000000,8.000000,15.000000,18.400000,0.078000,21.000000,0.000000
25%,1.000000,100.000000,64.000000,25.250000,114.000000,27.800000,0.256000,24.000000,0.000000
50%,3.000000,116.500000,72.069808,28.893519,135.921311,31.750000,0.349000,29.000000,0.000000
75%,5.000000,136.750000,80.000000,31.000000,135.921311,35.075000,0.551250,40.000000,1.000000
max,13.000000,197.000000,104.000000,56.000000,285.000000,49.700000,1.191000,66.000000,1.000000


# 3. Selección, validación y evaluación de modelos
Para poder realizar la validación del modelo, se utiliza el método de k-folding, dado que se cuenta con una cantidad de datos pequeña. Este algoritmo es implementado por la función **grid search**, que busca optimizar la métrica elegida (recall) variando los hiper parámetros del modelo. Luego, se entrena al modelo con los hiper parámetros que resultan de este análisis.

In [12]:
from sklearn.model_selection import GridSearchCV

In [23]:
from sklearn.metrics import recall_score

In [13]:
import itertools

## 3.1. Hiper parámetros

## 3.2. Modelo utilizando Gaussian Naive Bayes

In [14]:
from src.gaussian_naive_bayes import BinaryGaussianNaiveBayes

In [39]:
%%time

# Declaring the hiper parameters and their values for the GridSearchCV
# to search the best model according to the performance
parameters = {
    'std_smoothing': [0, 0.001, 0.01, 0.1, 1],
    'std_correction': [False, True],
    'filter_variables': list(itertools.product([True], repeat=8)),
}

# Estimator or model
estimator = BinaryGaussianNaiveBayes()

# Search the best model
grid = GridSearchCV(estimator, parameters, cv=10, scoring='recall', n_jobs=-1)
grid.fit(x_train, y_train)

Wall time: 7.53 s


GridSearchCV(cv=10, estimator=BinaryGaussianNaiveBayes(), n_jobs=-1,
             param_grid={'filter_variables': [(True, True, True, True, True,
                                               True, True, True)],
                         'std_correction': [False, True],
                         'std_smoothing': [0, 0.001, 0.01, 0.1, 1]},
             scoring='recall')

In [40]:
print(grid.best_params_)

{'filter_variables': (True, True, True, True, True, True, True, True), 'std_correction': False, 'std_smoothing': 0}


In [41]:
print(grid.best_score_)

0.6319326555797143


In [42]:
# Train the found model with the complete train set
classifier = BinaryGaussianNaiveBayes(
    std_smoothing=grid.best_params_['std_smoothing'], 
    std_correction=grid.best_params_['std_correction'], 
    filter_variables=grid.best_params_['filter_variables']
)
classifier.fit(x_train, y_train)

In [43]:
# Predictions using the test dataset and computing the score
predictions = classifier.predict(x_test)
score = recall_score(y_test, predictions)

In [46]:
print(f'Score of the model {np.round(score, 3)}')

Score of the model 0.653


In [47]:
from sklearn.naive_bayes import GaussianNB

# Create and train the model
c = GaussianNB()
c.fit(x_train, y_train)

# Predict and compute score
p = c.predict(x_test)
s = recall_score(y_test, p)

# Show the resulting score
print(f'Score of the model {np.round(s, 3)}')

Score of the model 0.653


## 3.3. Modelo utilizando Kernel Density Estimator

In [48]:
from src.kde_naive_bayes import BinaryKDENaiveBayes

In [107]:
%%time

# Declaring the hiper parameters and their values for the GridSearchCV
# to search the best model according to the performance
parameters = {
    'kernel': ['exponential', 'tophat', 'linear', 'cosine', 'gaussian', 'epanechnikov'],
    'bandwidth': [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10],
    'filter_variables': list(itertools.product([True, False], repeat=8)),
}

# Estimator or model
estimator = BinaryKDENaiveBayes()

# Search the best model
grid = GridSearchCV(estimator, parameters, cv=10, scoring='recall', n_jobs=-1)
grid.fit(x_train, y_train)

Wall time: 10.5 s


GridSearchCV(cv=10, estimator=BinaryKDENaiveBayes(), n_jobs=-1,
             param_grid={'bandwidth': [1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1,
                                       1, 10],
                         'filter_variables': [(True, True, True, True, True,
                                               True, True, True)],
                         'kernel': ['exponential', 'tophat', 'linear', 'cosine',
                                    'gaussian', 'epanechnikov']},
             scoring='recall')

In [108]:
print(grid.best_params_)

{'bandwidth': 1, 'filter_variables': (True, True, True, True, True, True, True, True), 'kernel': 'exponential'}


In [109]:
print(grid.best_score_)

0.5722334267040149


In [110]:
# Train the found model with the complete train set
classifier = BinaryKDENaiveBayes(
    kernel=grid.best_params_['kernel'], 
    bandwidth=grid.best_params_['bandwidth'], 
    filter_variables=grid.best_params_['filter_variables']
)
classifier.fit(x_train, y_train)

In [111]:
# Predictions using the test dataset and computing the score
predictions = classifier.predict(x_test)
score = recall_score(y_test, predictions)

In [112]:
print(f'Score of the model {np.round(score, 3)}')

Score of the model 0.592
